- 参照
    - [RAGコンペ参加記 (raggle)](https://qiita.com/ctc-j-ikai/items/9980f6a1c11ef444ba4d)

# Import

In [19]:
!pip install ragas==0.1.14
!pip install nest-asyncio==1.6.0

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.1/40.1 MB 9.9 MB/s eta 0:00:00:00:0100:01
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.12.0
    Uninstalling fsspec-2024.12.0:
      Successfully uninstalled fsspec-2024.12.0


In [25]:
import os
import json
import os
import sys
import time
from typing import Callable, Dict, List, Any
from dotenv import load_dotenv
import nest_asyncio
import pdfplumber
import requests

# from ragas.testset.generator import TestsetGenerator
# from ragas.testset.evolutions import simple, reasoning, multi_context
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain.schema import Document
from langchain.text_splitter import CharacterTextSplitter
from langchain_openai import ChatOpenAI
import pandas as pd
from langsmith import Client

# .envファイルを読み込む
load_dotenv()

os.environ["OPENAI_API_KEY"] = os.environ.get("OPENAI_API_KEY")
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_ENDPOINT"] = "https://api.smith.langchain.com"
os.environ["LANGCHAIN_API_KEY"] = os.environ.get("LANGCHAIN_API_KEY")
os.environ["LANGCHAIN_PROJECT"] = "rag-rohto"

In [23]:
pdf_file_urls = [
    "https://storage.googleapis.com/gg-raggle-public/competitions/617b10e9-a71b-4f2a-a9ee-ffe11d8d64ae/dataset/Financial_Statements_2023.pdf",
    "https://storage.googleapis.com/gg-raggle-public/competitions/617b10e9-a71b-4f2a-a9ee-ffe11d8d64ae/dataset/Hada_Labo_Gokujun_Lotion_Overview.pdf",
    "https://storage.googleapis.com/gg-raggle-public/competitions/617b10e9-a71b-4f2a-a9ee-ffe11d8d64ae/dataset/Shibata_et_al_Research_Article.pdf",
    "https://storage.googleapis.com/gg-raggle-public/competitions/617b10e9-a71b-4f2a-a9ee-ffe11d8d64ae/dataset/V_Rohto_Premium_Product_Information.pdf",
    "https://storage.googleapis.com/gg-raggle-public/competitions/617b10e9-a71b-4f2a-a9ee-ffe11d8d64ae/dataset/Well-Being_Report_2024.pdf",
]

# オフライン評価

## 評価用のテストデータの準備

In [12]:
from langsmith import Client

dataset_name = "rag-rohto"

client = Client()

if client.has_dataset(dataset_name=dataset_name):
    client.delete_dataset(dataset_name=dataset_name)

dataset = client.create_dataset(dataset_name=dataset_name)

In [2]:
valid_data = pd.read_csv("/root/git_work/90_adhoc/RagRohtoCompetition/dataset/validation.csv")
valid_data.head()

,query,context,groundtruth
0,肌ラボ 極潤ヒアルロン液の成分は？,4種類のヒアルロン酸（加水分解ヒアルロン酸、アセチルヒアルロン酸Na、乳酸球菌／ヒアルロン酸...,4種類のヒアルロン酸が配合されています。
1,極潤ヒアルロン液の特徴は？,弱酸性。無香料・無着色・オイルフリー・アルコール（エタノール）フリー・パラベンフリー。,弱酸性で、無香料・無着色・オイルフリー・アルコールフリー・パラベンフリーです。
2,極潤ヒアルロン液の使用上の注意は？,使用中、または使用後に赤みやかゆみなどの異常が現れた場合は、使用を中止してください。,使用中や使用後に異常が現れた場合は使用を中止してください。
3,極潤ヒアルロン液は誰が使用可能ですか？,老若男女、誰でも使用できます。,老若男女、誰でも使用可能です。
4,極潤ヒアルロン液の容器に関する特長は？,容器には環境に配慮したバイオマス原料が一部使用されています。,環境に配慮したバイオマス原料を一部使用しています。


In [13]:
# 空のリストを初期化
inputs = []
outputs = []
# metadatas = []

# 各レコードを処理してリストに追加
for _, row in valid_data.iterrows():
    # Inputs
    inputs.append(
        {
            "question": row["query"],
        }
    )

    # Outputs
    outputs.append(
        {
            "contexts": row["context"],
            "ground_truth": row["groundtruth"],
        }
    )


# 結果を確認
print("Inputs:", inputs[:2])  # 最初の2件を確認
print("Outputs:", outputs[:2])

Inputs: [{'question': '肌ラボ 極潤ヒアルロン液の成分は？'}, {'question': '極潤ヒアルロン液の特徴は？'}]
Outputs: [{'contexts': '4種類のヒアルロン酸（加水分解ヒアルロン酸、アセチルヒアルロン酸Na、乳酸球菌／ヒアルロン酸発酵液、ヒアルロン酸Na）を配合。', 'ground_truth': '4種類のヒアルロン酸が配合されています。'}, {'contexts': '弱酸性。無香料・無着色・オイルフリー・アルコール（エタノール）フリー・パラベンフリー。', 'ground_truth': '弱酸性で、無香料・無着色・オイルフリー・アルコールフリー・パラベンフリーです。'}]


In [16]:
client.create_examples(
    inputs=inputs,
    outputs=outputs,
    # metadata=metadatas,
    # dataset_id=dataset.id,
    dataset_name=dataset_name,
)

In [32]:
dataset

Dataset(name='rag-rohto', description=None, data_type=<DataType.kv: 'kv'>, id=UUID('686942db-401e-4aa9-9cac-a44fbc1ab041'), created_at=datetime.datetime(2025, 1, 11, 8, 57, 5, 315373, tzinfo=datetime.timezone.utc), modified_at=datetime.datetime(2025, 1, 11, 8, 57, 5, 315373, tzinfo=datetime.timezone.utc), example_count=0, session_count=0, last_session_start_time=None, inputs_schema=None, outputs_schema=None)

## LangSmith と Ragas を使ったオフライン評価の実装

In [26]:
# 戻り値として質問に対する回答を返却してください。
def download_and_load_pdfs(urls: list) -> list:
    """
    PDFファイルをダウンロードして読み込む関数

    Args:
        urls (list): PDFファイルのURLリスト

    Returns:
        documents (list): PDFファイルのテキストデータを含むDocumentオブジェクトのリスト

    Raises:
        Exception: ダウンロードまたは読み込みに失敗した場合に発生する例外

    Examples:
        >>> urls = ["https://example.com/example.pdf"]
        >>> download_and_load_pdfs(urls)
        [Document(page_content="...", metadata={"source": "https://example.com/example.pdf"})]
    """
    try:

        def download_pdf(url, save_path):
            response = requests.get(url)
            if response.status_code == 200:
                with open(save_path, "wb") as f:
                    f.write(response.content)
            else:
                raise Exception(f"Failed to download {url}")

        documents = []

        for i, url in enumerate(urls):
            tmp_path = f"pdf_{i}.pdf"
            download_pdf(url, tmp_path)

            with pdfplumber.open(tmp_path) as pdf:
                full_text = ""
                for page in pdf.pages:
                    text = page.extract_text()
                    if text:
                        full_text += text + "\n"

                documents.append(Document(page_content=full_text, metadata={"source": url}))
        return documents
    except Exception as e:
        raise Exception(f"Error reading {url}: {e}")


def create_vectorstore(docs: list) -> Chroma:
    """
    テキストデータからベクトルストアを生成する関数

    Args:
        docs (list): Documentオブジェクトのリスト

    Returns:
        vectorstore (Chroma): ベクトルストア

    Raises:
        Exception: ベクトルストアの生成に失敗した場合に発生する例外

    Examples:
        >>> docs = [Document(page_content="...", metadata={"source": "https://example.com/example.pdf"})]
        >>> create_vectorstore(docs)
        Chroma(...)
    """
    try:
        text_splitter = CharacterTextSplitter(
            separator="\n",
            chunk_size=1000,
        )
        splitted_docs = []
        for doc in docs:
            chunks = text_splitter.split_text(doc.page_content)
            for chunk in chunks:
                splitted_docs.append(Document(page_content=chunk, metadata=doc.metadata))

        embedding_function = OpenAIEmbeddings()

        vectorstore = Chroma.from_documents(
            splitted_docs,
            embedding_function,
        )
        return vectorstore
    except Exception as e:
        raise Exception(f"Error creating vectorstore: {e}")


docs = download_and_load_pdfs(pdf_file_urls)
# db = create_vectorstore(docs)
# retriever = db.as_retriever()

In [20]:
from typing import Any

from langchain_core.embeddings import Embeddings
from langchain_core.language_models import BaseChatModel
from langsmith.schemas import Example, Run
from ragas.embeddings import LangchainEmbeddingsWrapper
from ragas.llms import LangchainLLMWrapper
from ragas.metrics.base import Metric, MetricWithEmbeddings, MetricWithLLM


class RagasMetricEvaluator:
    def __init__(self, metric: Metric, llm: BaseChatModel, embeddings: Embeddings):
        self.metric = metric

        # LLMとEmbeddingsをMetricに設定
        if isinstance(self.metric, MetricWithLLM):
            self.metric.llm = LangchainLLMWrapper(llm)
        if isinstance(self.metric, MetricWithEmbeddings):
            self.metric.embeddings = LangchainEmbeddingsWrapper(embeddings)

    def evaluate(self, run: Run, example: Example) -> dict[str, Any]:
        context_strs = [doc.page_content for doc in run.outputs["contexts"]]

        # Ragasの評価メトリクスのscoreメソッドでスコアを算出
        score = self.metric.score(
            {
                "question": example.inputs["question"],
                "answer": run.outputs["answer"],
                "contexts": context_strs,
                "ground_truth": example.outputs["ground_truth"],
            },
        )
        return {"key": self.metric.name, "score": score}

/root/anaconda3/envs/ragrohto/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [21]:
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from ragas.metrics import answer_relevancy, context_precision

metrics = [context_precision, answer_relevancy]

llm = ChatOpenAI(model="gpt-4o", temperature=0)
embeddings = OpenAIEmbeddings(model="text-embedding-3-small")

evaluators = [RagasMetricEvaluator(metric, llm, embeddings).evaluate for metric in metrics]

In [27]:
from langchain_chroma import Chroma
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(model="text-embedding-3-small")
db = Chroma.from_documents(docs, embeddings)

In [28]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableParallel, RunnablePassthrough
from langchain_openai import ChatOpenAI

prompt = ChatPromptTemplate.from_template(
    '''\
以下の文脈だけを踏まえて質問に回答してください。

文脈: """
{context}
"""

質問: {question}
'''
)

model = ChatOpenAI(model="gpt-4o-mini", temperature=0)

retriever = db.as_retriever()

chain = RunnableParallel(
    {
        "question": RunnablePassthrough(),
        "context": retriever,
    }
).assign(answer=prompt | model | StrOutputParser())

In [29]:
def predict(inputs: dict[str, Any]) -> dict[str, Any]:
    question = inputs["question"]
    output = chain.invoke(question)
    return {
        "contexts": output["context"],
        "answer": output["answer"],
    }

In [31]:
from langsmith.evaluation import evaluate

evaluate(
    predict,
    data="rag-rohto",
    evaluators=evaluators,
)

View the evaluation results for experiment: 'crushing-cheek-49' at:
https://smith.langchain.com/o/95cad935-bb69-4a11-a34c-beb1e1749d98/datasets/686942db-401e-4aa9-9cac-a44fbc1ab041/compare?selectedSessions=7a9554ec-8087-4bca-9c47-cdd84a782f67




0it [00:00, ?it/s]Error running evaluator <DynamicRunEvaluator evaluate> on run bd7850b7-4c43-43b8-9413-ba62c6cb0f05: RuntimeError("There is no current event loop in thread 'ThreadPoolExecutor-1_0'.")
Traceback (most recent call last):
  File "/root/anaconda3/envs/ragrohto/lib/python3.11/site-packages/langsmith/evaluation/_runner.py", line 1404, in _run_evaluators
    evaluator_response = evaluator.evaluate_run(
                         ^^^^^^^^^^^^^^^^^^^^^^^
  File "/root/anaconda3/envs/ragrohto/lib/python3.11/site-packages/langsmith/evaluation/evaluator.py", line 331, in evaluate_run
    result = self.func(
             ^^^^^^^^^^
  File "/root/anaconda3/envs/ragrohto/lib/python3.11/site-packages/langsmith/run_helpers.py", line 617, in wrapper
    raise e
  File "/root/anaconda3/envs/ragrohto/lib/python3.11/site-packages/langsmith/run_helpers.py", line 614, in wrapper
    function_result = run_container["context"].run(func, *args, **kwargs)
                      ^^^^^^^^^^^^^^^^^^^^

,inputs.question,outputs.contexts,outputs.answer,error,reference.contexts,reference.ground_truth,feedback.evaluate,execution_time,example_id,id,outputs.output
0,極潤ヒアルロン液は敏感肌でも使用できますか？,[page_content='2024/12/06 8:55 肌ラボ 極潤ヒアルロン液 | ...,はい、極潤ヒアルロン液は敏感肌の方でも使用できるように設計されています。製品は弱酸性で、無香...,None,敏感肌の方の協力によるパッチテスト済み。ただし、すべての方に刺激が起こらないわけではありません。,敏感肌の方も使用可能ですが、すべての方に刺激が起こらないわけではありません。,None,14.571535,c7d32acf-d645-43a8-bb2c-da38a1f015c3,bd7850b7-4c43-43b8-9413-ba62c6cb0f05,NaN
1,極潤ヒアルロン液が推奨されている肌タイプは？,NaN,NaN,"RateLimitError(""Error code: 429 - {'error': {'...",すべての肌タイプに適応可能です。,すべての肌タイプに適しています。,None,28.611404,2f2698a4-f58f-4de8-86fb-afd20c6a0a07,807c2493-ece8-4eab-89e8-f86c13115431,NaN
2,極潤ヒアルロン液は航空法の対象となりますか？,NaN,NaN,"RateLimitError(""Error code: 429 - {'error': {'...",本品は、航空法で定める航空危険物に該当しません。,該当しません。,None,28.634863,7975bec9-43cc-44ad-b182-453d2ac361d7,bfc096b7-0dcf-44dc-9d85-f520f619c4f8,NaN
3,極潤ヒアルロン液が環境に配慮している点は？,NaN,NaN,"RateLimitError(""Error code: 429 - {'error': {'...",容器に環境に配慮したバイオマス原料を一部使用しています。,バイオマス原料を一部使用した容器を採用しています。,None,28.781769,8fb0b9ca-c849-4e1f-8248-374888a278e8,a546b198-220b-41c7-b73d-6b6b3f792d58,NaN
4,肌ラボ 極潤ヒアルロン液が含むヒアルロン酸の種類は？,NaN,NaN,"RateLimitError(""Error code: 429 - {'error': {'...",4種類のヒアルロン酸（加水分解ヒアルロン酸、アセチルヒアルロン酸Na、乳酸球菌／ヒアルロン酸...,加水分解ヒアルロン酸、アセチルヒアルロン酸Na、乳酸球菌／ヒアルロン酸発酵液、ヒアルロン酸N...,None,28.779880,22b8c0dc-9d43-43f2-b82a-94d0d05e5ccc,519706f0-8a25-403c-9a17-9aede0f2a888,NaN
5,極潤ヒアルロン液の肌への効果は？,NaN,NaN,"RateLimitError(""Error code: 429 - {'error': {'...",たっぷりうるおい、もちもちした肌へ導きます。,肌にたっぷりうるおいを与え、もちもち肌にします。,None,28.836716,fbf5d0ed-aea9-4fae-a222-daacb56289b8,374c67c4-b29b-48a2-a9a3-2206cc22eefb,NaN
6,極潤ヒアルロン液の詰め替え用の内容量は？,NaN,NaN,BadRequestError('Error code: 400 - {\'error\':...,詰め替え用の内容量は170mLです。,詰め替え用の内容量は170mLです。,None,28.846703,e05da635-359d-4d3a-9718-6442c9d1f45a,0a195259-e136-4994-94b0-7ecb9462f5b3,NaN
7,極潤ヒアルロン液の容器に関する特長は？,NaN,NaN,"RateLimitError(""Error code: 429 - {'error': {'...",容器には環境に配慮したバイオマス原料が一部使用されています。,環境に配慮したバイオマス原料を一部使用しています。,None,28.830553,402cf295-be1c-4690-9e7d-39a313815cbc,a392e966-2326-4e39-9fbb-c970374dd28e,NaN
8,極潤ヒアルロン液は子供にも使用できますか？,NaN,NaN,"RateLimitError(""Error code: 429 - {'error': {'...",老若男女、誰でも使用可能です。,老若男女が使用可能です。,None,28.875581,293c2373-a893-40fe-9529-e61b913e2143,561f11d1-6c7c-4e34-aa05-b61efbf7f118,NaN
9,極潤ヒアルロン液の使用感はどのようなものですか？,NaN,NaN,"RateLimitError(""Error code: 429 - {'error': {'...",やさしいとろみで、吸いつくような使用感です。,やさしいとろみで吸いつくような使用感です。,None,28.880445,76d145e9-d445-489f-ac64-fc4c1f2becb3,bd819f7e-ea59-42a9-94de-551ba4bd17f8,NaN
